**Cell to store the keys**

In [ ]:
import os

api_key = input("Please enter your OPENAI API key: ")

os.environ["OPENAI_API_KEY"] = api_key

print("OPENAI_API_KEY has been set!")

github_token=input("Please enter your Github PAT: ")

os.environ["GITHUB_TOKEN"]= github_token

print("Github PAT has been set!")

Please enter your OPENAI API key: a
OPENAI_API_KEY has been set!
Please enter your Github PAT: b
Github PAT has been set!


Mount the drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Installing the dependencies


In [ ]:
!pip install langchain
!pip install openai
!pip install tiktoken
!pip install faiss-gpu
!pip install langchain_experimental
!pip install "langchain[docarray]"
!pip install anthropic
!pip install ipython
!pip install langchain-openai

Import

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.indexes import VectorstoreIndexCreator
from langchain_experimental.agents.agent_toolkits.csv.base import create_csv_agent
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory
import tiktoken
import os
import json
import anthropic
import os
import requests
import openai

The below cell will generate the cards, click on the github repo (it copies the link to clipboard) that you want to chat with.

In [ ]:
from IPython.display import HTML, display, Javascript
import openai
import os
import requests

global selected_url

class OpenAIChat:

    def __init__(self, api_key):
        self.client = openai.OpenAI(api_key=api_key)

    def generate_keywords(self, prompt):
        chat_completion = self.client.chat.completions.create(
            messages=[
                {"role": "system", "content": "Return 2 or 3 keywords for the following prompt that can be used to search GitHub repos. Don't enumerate it, just display them in X Y or X Y Z format"},
                {"role": "user", "content": prompt},
            ],
            model="gpt-3.5-turbo",
        )

        assistant_message_content = chat_completion.choices[0].message.content

        return assistant_message_content

def prompt_user_for_query():
    user_query = input("Enter your query: ")
    return user_query

def search_github_repositories(token, keywords):
    base_url = "https://api.github.com/search/repositories"
    headers = {"Authorization": f"token {token}"}
    params = {"q": keywords}

    response = requests.get(base_url, headers=headers, params=params)

    if response.status_code == 200:
        return response.json()["items"]
    else:
        print(f"Error: {response.status_code}")
        return None

def generate_repo_cards(repositories):
    global selected_url

    cards_html = ""
    for repo in repositories:
        card_html = f"""
            <div onclick="copyToClipboard('{repo['html_url']}')" style="cursor: pointer; border: 1px solid #ddd; padding: 10px; margin: 10px; border-radius: 5px;">
                <h3>{repo['name']}</h3>
                <p><strong>Description:</strong> {repo['description']}</p>
                <p><strong>URL:</strong> <a href="{repo['html_url']}" target="_blank">{repo['html_url']}</a></p>
            </div>
        """
        cards_html += card_html

    script = """
        <script>
            function copyToClipboard(url) {
                var tempInput = document.createElement('input');
                tempInput.value = url;
                document.body.appendChild(tempInput);
                tempInput.select();
                document.execCommand('copy');
                document.body.removeChild(tempInput);

                var selected_url_display = document.getElementById('selected_url_display');
                selected_url_display.innerHTML = "Selected URL: " + url;

                console.log("Selected URL:", url);
            }
        </script>
    """

    display(HTML(cards_html + script))

if __name__ == "__main__":
    openai_api_key = os.environ.get("OPENAI_API_KEY")
    github_token = os.environ.get("GITHUB_TOKEN")

    user_query = prompt_user_for_query()

    openai_chat = OpenAIChat(api_key=openai_api_key)

    generated_keywords = openai_chat.generate_keywords(user_query)

    repositories = search_github_repositories(github_token, generated_keywords)

    if repositories:
        generate_repo_cards(repositories)
        display(HTML('<div id="selected_url_display"></div>'))
    else:
        print("No repositories found.")

Enter your query: python repo for ros


This deletes the existing repo. Run this when you want to use a new repo as context.

In [ ]:
import shutil

directory_path = '/content/your-repo'

shutil.rmtree(directory_path)

print(f"Directory '{directory_path}' has been deleted.")


Directory '/content/your-repo' has been deleted.


Saves the repo as a single json

In [ ]:
import subprocess
print(selected_url)
def is_code_file(filename):
    code_extensions = ['.py', '.java', '.cpp', '.html', '.css', '.js', '.txt', '.ipynb']
    return any(filename.endswith(ext) for ext in code_extensions)

def parse_project_folder(folder_path, output_json_path):
    code_contents = {}

    for root, dirs, files in os.walk(folder_path):
        for file_name in files:
            file_path = os.path.join(root, file_name)

            if not is_code_file(file_name):
                continue

            with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
                content = file.read()
                code_contents[file_name] = content
                print(code_contents)

    with open(output_json_path, 'w') as json_file:
        json.dump(code_contents, json_file, indent=4)

if __name__ == "__main__":


    github_repo_url = input("Paste the repo link")
    colab_repo_path = "/content/your-repo"

    subprocess.run(["git", "clone", github_repo_url, colab_repo_path])


    project_folder = colab_repo_path
    output_json_file = "/content/code_content2.json"

    parse_project_folder(project_folder, output_json_file)
    print(f"Code contents saved to {output_json_file}")

 
Paste the repo linkhttps://github.com/microsoft/Ros-Orchestration
{'index.html': '<!-- Copyright (c) Microsoft Corporation. All rights reserved.\n     Licensed under the MIT License. -->\n\n<!DOCTYPE html>\n<html lang="en">\n  <head>\n    <meta charset="utf-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">\n    <meta name="theme-color" content="#000000">\n    <!--\n      manifest.json provides metadata used when your web app is added to the\n      homescreen on Android. See https://developers.google.com/web/fundamentals/engage-and-retain/web-app-manifest/\n    -->\n    <link rel="manifest" href="%PUBLIC_URL%/manifest.json">\n    <link rel="shortcut icon" href="%PUBLIC_URL%/favicon.ico">\n\n    <!-- Latest react-bootstrap stylesheet -->\n    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" integrity="sha384-BVYiiSIFeK1dGmJRAkycuHAHRg32OmUcww7on3RYdg4Va+PmSTsz/K68vbdEjh4u" crossorigin="anon

Converts the JSON to txt

In [ ]:

def json_to_txt(json_file_path, txt_file_path):
    with open(json_file_path, 'r') as json_file:
        data = json.load(json_file)

    with open(txt_file_path, 'w') as txt_file:
        for key, value in data.items():
            txt_file.write(f"{key}:\n{value}\n\n")

if __name__ == "__main__":
    json_file_path = "/content/code_content2.json"
    txt_file_path = "output2.txt"

    json_to_txt(json_file_path, txt_file_path)
    print(f"JSON converted to {txt_file_path}")

JSON converted to output2.txt


In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter


Chunking the txt

In [ ]:
txt_file_path = '/content/output2.txt'
loader = TextLoader(file_path=txt_file_path, encoding="utf-8")
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
text_splitter.split_documents(data)[:5]

[Document(page_content='index.html:\n<!-- Copyright (c) Microsoft Corporation. All rights reserved.', metadata={'source': '/content/output2.txt'}),
 Document(page_content='Licensed under the MIT License. -->', metadata={'source': '/content/output2.txt'}),
 Document(page_content='<!DOCTYPE html>\n<html lang="en">\n  <head>\n    <meta charset="utf-8">', metadata={'source': '/content/output2.txt'}),
 Document(page_content='<meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">', metadata={'source': '/content/output2.txt'}),
 Document(page_content='<meta name="theme-color" content="#000000">\n    <!--', metadata={'source': '/content/output2.txt'})]

In [ ]:
data

[Document(page_content='index.html:\n<!-- Copyright (c) Microsoft Corporation. All rights reserved.\n     Licensed under the MIT License. -->\n\n<!DOCTYPE html>\n<html lang="en">\n  <head>\n    <meta charset="utf-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">\n    <meta name="theme-color" content="#000000">\n    <!--\n      manifest.json provides metadata used when your web app is added to the\n      homescreen on Android. See https://developers.google.com/web/fundamentals/engage-and-retain/web-app-manifest/\n    -->\n    <link rel="manifest" href="%PUBLIC_URL%/manifest.json">\n    <link rel="shortcut icon" href="%PUBLIC_URL%/favicon.ico">\n\n    <!-- Latest react-bootstrap stylesheet -->\n    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" integrity="sha384-BVYiiSIFeK1dGmJRAkycuHAHRg32OmUcww7on3RYdg4Va+PmSTsz/K68vbdEjh4u" crossorigin="anonymous">\n    <link rel="stylesheet" href="https

Embedding done to store the words as vectors

In [ ]:
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
print(f"Embedding length: {len(embedded_query)}")

Embedding length: 1536


In [ ]:
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(data, embedding=embeddings)

In [ ]:

llm = ChatOpenAI(temperature=0.7, model_name="gpt-3.5-turbo")
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)


conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    memory=memory
)


Run it and ask the repo all the questions

In [ ]:
query = input("Chat with your repo ")
result = conversation_chain({"question": query})
answer = result["answer"]
answer

Chat with your repo what is the core framework used


'The core framework used in this repository for controlling the robots and managing their tasks is ROS (Robot Operating System).'